# U-Index Visualizations

Interactive visualizations demonstrating how the U-index differentiates researchers with similar h-indices but different leadership profiles.

This notebook provides interactive Plotly charts. For static figures used in publications, run the script: `python u_index_visualizations.py`

In [1]:
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def hex_to_rgba(hex_color: str, alpha: float) -> str:
    """Convert hex color to rgba string."""
    hex_color = hex_color.lstrip("#")
    r, g, b = int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)
    return f"rgba({r}, {g}, {b}, {alpha})"


# Researcher profiles with synthetic data
RESEARCHERS = [
    {"name": "Dr. Early", "archetype": "Career Stage", "description": "5 years post-PhD, primarily first-author experimental work", "h": 12, "u": 10, "years": 5, "h_base_rate": 2.4, "collab_bonus": 0.4},
    {"name": "Dr. Midcareer", "archetype": "Career Stage", "description": "12 years in, mix of independent and supervised research", "h": 25, "u": 18, "years": 12, "h_base_rate": 1.5, "collab_bonus": 0.58},
    {"name": "Dr. Senior", "archetype": "Career Stage", "description": "25 years experience, leads large lab with heavy supervision", "h": 45, "u": 22, "years": 25, "h_base_rate": 0.88, "collab_bonus": 0.92},
    {"name": "Dr. Independent", "archetype": "Leadership Style", "description": "Runs small lab, still does own experiments", "h": 20, "u": 18, "years": 15, "h_base_rate": 1.2, "collab_bonus": 0.13},
    {"name": "Dr. Collaborative", "archetype": "Leadership Style", "description": "Hub in large consortiums, many middle-author papers", "h": 35, "u": 8, "years": 15, "h_base_rate": 0.53, "collab_bonus": 1.8},
    {"name": "Dr. Balanced", "archetype": "Leadership Style", "description": "Equal leadership and collaboration contributions", "h": 28, "u": 14, "years": 15, "h_base_rate": 0.93, "collab_bonus": 0.93},
    {"name": "Dr. Consortium", "archetype": "Edge Case", "description": "Almost entirely middle-author consortium positions", "h": 40, "u": 3, "years": 20, "h_base_rate": 0.15, "collab_bonus": 1.85},
    {"name": "Dr. Solo", "archetype": "Edge Case", "description": "Single-author theoretician, all papers are first/last", "h": 15, "u": 15, "years": 20, "h_base_rate": 0.75, "collab_bonus": 0.0},
]

## H-index vs U-index Scatter Plot

This plot shows the relationship between h-index (total impact) and U-index (leadership impact) for eight simulated researcher profiles.

- **Diagonal line**: Represents U = h (all papers are first/last authored)
- **Distance below diagonal**: Impact accumulated through middle-author positions
- **Points colored by category**: Career Stage, Leadership Style, Edge Cases

In [3]:
def create_scatter_plot() -> go.Figure:
    """Create scatter plot showing h vs U for all researchers."""
    fig = go.Figure()

    max_val = max(r["h"] for r in RESEARCHERS) + 5
    fig.add_trace(go.Scatter(
        x=[0, max_val], y=[0, max_val],
        mode="lines", name="U = h (perfect leadership)",
        line=dict(color="#999", width=1, dash="dash"), hoverinfo="skip",
    ))

    archetype_colors = {"Career Stage": "#3b82f6", "Leadership Style": "#22c55e", "Edge Case": "#f97316"}

    for archetype in ["Career Stage", "Leadership Style", "Edge Case"]:
        researchers = [r for r in RESEARCHERS if r["archetype"] == archetype]
        fig.add_trace(go.Scatter(
            x=[r["h"] for r in researchers], y=[r["u"] for r in researchers],
            mode="markers+text", name=archetype,
            marker=dict(size=12, color=archetype_colors[archetype]),
            text=[r["name"].replace("Dr. ", "") for r in researchers],
            textposition="top center", textfont=dict(size=10),
            customdata=[r["description"] for r in researchers],
            hovertemplate="<b>%{text}</b><br>h-index: %{x}<br>U-index: %{y}<extra></extra>",
        ))

    fig.update_layout(
        title={"text": "H-index vs U-index: Leadership Impact Profile"},
        xaxis_title="h-index (total impact)", yaxis_title="U-index (leadership impact)",
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
        xaxis=dict(range=[0, max_val], constrain="domain"),
        yaxis=dict(range=[0, max_val], scaleanchor="x", scaleratio=1),
        width=600, height=600,
    )
    return fig

In [4]:
create_scatter_plot().show()

## Stacked Bar Chart: Leadership vs Collaboration

This chart decomposes each researcher's h-index into:
- **Blue (bottom)**: U-index (leadership impact from first/last author papers)
- **Gray (top)**: Collaboration contribution (h - U, from middle-author positions)

The total bar height equals the h-index.

In [5]:
def create_comparison_chart() -> go.Figure:
    """Create grouped bar chart comparing h and U across all researchers."""
    sorted_researchers = sorted(RESEARCHERS, key=lambda r: r["h"], reverse=True)

    names = [r["name"] for r in sorted_researchers]
    h_values = [r["h"] for r in sorted_researchers]
    u_values = [r["u"] for r in sorted_researchers]
    ratios = [f"U/h = {u/h:.0%}" for h, u in zip(h_values, u_values)]
    collab_values = [h - u for h, u in zip(h_values, u_values)]

    fig = go.Figure()
    fig.add_trace(go.Bar(
        name="U-index (leadership)", x=names, y=u_values,
        marker_color="rgba(0, 115, 200, 1)",
        hovertemplate="<b>%{x}</b><br>U-index: %{y}<extra></extra>",
    ))
    fig.add_trace(go.Bar(
        name="Collaboration contribution", x=names, y=collab_values,
        marker_color="rgba(107, 114, 128, 0.5)",
        customdata=h_values,
        hovertemplate="<b>%{x}</b><br>h-index: %{customdata}<br>Collaboration: %{y}<extra></extra>",
    ))

    fig.update_layout(
        title={"text": "H-index vs U-index Across Researcher Profiles"},
        xaxis_title="Researcher", yaxis_title="Index Value",
        barmode="stack",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        template="plotly_white", width=800, height=500,
    )

    for i, (name, ratio) in enumerate(zip(names, ratios)):
        fig.add_annotation(x=name, y=h_values[i] + 2, text=ratio, showarrow=False, font=dict(size=10, color="#666"))

    return fig

In [6]:
create_comparison_chart().show()

## Career Trajectories

These plots show how h-index and U-index diverge over a researcher's career:
- **Solid line**: U-index (leadership impact)
- **Dashed line**: h-index (total impact)
- **Shaded area**: Gap representing collaboration contributions

In [7]:
def generate_trajectory(researcher: dict, seed: int = 42) -> list[tuple[int, int, int]]:
    """Generate a career trajectory showing h and U values over time."""
    random.seed(seed + hash(researcher["name"]))
    trajectory = [(0, 0, 0)]
    h_cumulative, u_cumulative = 0.0, 0.0
    h_base, collab = researcher["h_base_rate"], researcher["collab_bonus"]

    for year in range(1, researcher["years"] + 1):
        h_cumulative += h_base + collab + random.uniform(-0.2, 0.3)
        u_cumulative += h_base + random.uniform(-0.15, 0.25)
        h_cumulative = max(h_cumulative, trajectory[-1][1])
        u_cumulative = max(u_cumulative, trajectory[-1][2])
        u_cumulative = min(u_cumulative, h_cumulative)
        trajectory.append((year, int(h_cumulative), int(u_cumulative)))

    final_h, final_u = researcher["h"], researcher["u"]
    scale_h = final_h / max(trajectory[-1][1], 1)
    scale_u = final_u / max(trajectory[-1][2], 1)

    scaled = [(0, 0, 0)]
    for year, h, u in trajectory[1:]:
        scaled_h, scaled_u = int(h * scale_h), min(int(u * scale_u), int(h * scale_h))
        scaled.append((year, scaled_h, scaled_u))
    scaled[-1] = (researcher["years"], final_h, final_u)
    return scaled

In [8]:
def create_trajectory_grid() -> go.Figure:
    """Create 2x2 grid showing each researcher's trajectory in separate panes."""
    selected = ["Dr. Independent", "Dr. Collaborative", "Dr. Senior", "Dr. Solo"]
    colors = {"Dr. Independent": "#ef4444", "Dr. Collaborative": "#22c55e", "Dr. Senior": "#3b82f6", "Dr. Solo": "#a855f7"}

    fig = make_subplots(rows=2, cols=2, subplot_titles=selected, horizontal_spacing=0.1, vertical_spacing=0.15)
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

    for i, name in enumerate(selected):
        researcher = next(r for r in RESEARCHERS if r["name"] == name)
        color = colors[name]
        row, col = positions[i]
        trajectory = generate_trajectory(researcher)
        years = [t[0] for t in trajectory]
        h_vals = [t[1] for t in trajectory]
        u_vals = [t[2] for t in trajectory]

        fig.add_trace(go.Scatter(x=years, y=u_vals, mode="lines", name="U-index", line=dict(color=color, width=2), showlegend=(i == 0)), row=row, col=col)
        fig.add_trace(go.Scatter(x=years, y=h_vals, mode="lines", name="h-index", line=dict(color=color, width=2, dash="dash"), showlegend=(i == 0)), row=row, col=col)
        fig.add_trace(go.Scatter(x=years, y=u_vals, mode="none", fill="tozeroy", fillcolor=hex_to_rgba(color, 0.3), showlegend=False), row=row, col=col)
        fig.add_trace(go.Scatter(x=years + years[::-1], y=h_vals + u_vals[::-1], mode="none", fill="toself", fillcolor=hex_to_rgba(color, 0.1), showlegend=False), row=row, col=col)

    fig.update_layout(title={"text": "Career Trajectories by Researcher Profile"}, template="plotly_white", height=700, showlegend=False)
    fig.update_xaxes(title_text="Career Year")
    fig.update_yaxes(title_text="Index Value")
    return fig

In [9]:
create_trajectory_grid().show()

## Key Insights

| Researcher | h-index | U-index | U/h | Interpretation |
|------------|---------|---------|-----|----------------|
| Dr. Solo | 15 | 15 | 100% | All impact from leadership |
| Dr. Independent | 20 | 18 | 90% | Mostly leadership |
| Dr. Collaborative | 35 | 8 | 23% | Mostly collaboration |
| Dr. Consortium | 40 | 3 | 7.5% | Almost entirely collaboration |

The U-index serves as a **conservative lower bound** on leadership impact. Two researchers with identical h-indices can have vastly different U-indices, revealing whether their impact comes from leading research or participating in collaborations.